In [1]:
from music21 import *
environment.Environment()['musicxmlPath'] = r'D:\Program Files\MuseScore 3\bin\MuseScore3.exe'
environment.Environment()['musescoreDirectPNGPath'] = r'D:\Program Files\MuseScore 3\bin\MuseScore3.exe'

In [42]:


PitchClass = ['C','D','E','F','G','A','B']
TypeClass = ['64th','32nd','16th','eighth','quarter','half','whole']   
class StrNote:
    '''
     对音符的字符串信息进行保存
     分别存储pitch，octave ,accidental，并最终拼接起来
    '''
    def __init__(self):
        '''
            默认简谱中的do是C5
            因此是pitch下标为0
            中音的八度octave是第五个
            没有升降号accidental 
            默认是四分之一音符4
            默认没有附点
        '''
        self.pitch = 0
        self.octave = 5
        self.accidental = ''
        self.type = 4
        self.dot = 0
        
    def __init__(self,pitch):
        '''
          有参构造
        '''
        self.pitch = pitch-1
        self.octave = 5
        self.accidental = ''
        self.type = 4
        self.dot = 0
            
    def __add__(self, y):
        '音调pitch加一位'
        self.pitch = (self.pitch+y)%7
        return self
        
    def __sub__(self,y):
        '音调pitch减一位'
        self.pitch = (self.pitch+7-y)%7
        return self
    
    def setflat(self):
        '降音'
        self.accidental = 'b'
    
    def setsharp(self):
        '升音'
        self.accidental = '#'
    
    def setnormal(self):
        '变成普通音'
        self.accidental = ''
        
    def higher(self):
        '提高一个八度'
        self.octave += 1 
        
    def lower(self):
        '降低一个八度'
        self.octave -= 1 
    
    def longer(self):
        '时间加长'
        if self.type == 5 and self.dot == 0:
            self.dot = 1
        else:
            self.type += 1
            
    def shotter(self):
        '时间减短'
        self.type -= 1
    
    def adddot(self):
        '添加附点'
        self.dot += 1
        
    def getstr(self):
        '返回这个音的字符串'
        return str(PitchClass[self.pitch]) + str(self.octave) + str(self.accidental)
        
    
    def getnote(self):
        if self.pitch == -1:
            stopnote = note.Rest(1)
            stopnote.duration.type = TypeClass[self.type]
            stopnote.duration.dots = self.dot
            return stopnote
        '返回音符对象'
        thisnote = note.Note(str(PitchClass[self.pitch]) + str(self.octave) + str(self.accidental))
        thisnote.duration.type = TypeClass[self.type]
        thisnote.duration.dots = self.dot
        return thisnote
 
'''判断是数字还是符号'''
def isnum(input):
    try:
        int(input)
        return True
    except:
        return False
    
    

'''
Summay:
    将字符串简谱转换成音乐流
Parameters:
    music_str - 简谱字符串  str
    play_it - 是否播放      bool
Return：
    s - Music21格式的音乐流 stream
'''
def str2stream(music_str,play_it = True):
    #out_put = open('out.txt','w')
    notelist = []
    s = stream.Score(id='mainScore')
    '''
    lowerflag  -  低音标志()
    higherflag -  高音标志[]
    shotterflag - 短时标志<>
    longgerflag - 长时标志--
    
    '''
    lowerflag = False
    higherflag = False
    sharpflag = False
    shotterflag = 0
    measurenum = 1
    for each in music_str:
        #print(each)
        if isnum(each):
            midnote = StrNote(int(each))
            if lowerflag == True:
                midnote.lower()
            if higherflag == True:
                midnote.higher()
            if shotterflag != 0:
                for i in range(shotterflag):
                    midnote.shotter()
            if sharpflag == True:
                midnote.setsharp()
                #print(midnote)
                sharpflag = False
            notelist.append(midnote)
        else:
            if each == '(':
                lowerflag = True
            elif each == ')':
                lowerflag = False
            elif each == '[':
                higherflag = True 
            elif each == ']':
                higherflag = False 
            elif each == '<':
                shotterflag += 1 
            elif each == '>':
                shotterflag -= 1
            elif each == '.':
                notelist[len(notelist)-1].adddot()
            elif each == '-':
                notelist[len(notelist)-1].longer()
            elif each == '#':
                #sharpflag = True
                notelist[len(notelist)-1].setsharp()
            elif each == 'b':
                notelist[len(notelist)-1].setflat()
            elif each == '|':
                midstream = stream.Measure(number=measurenum)
                measurenum+=1
                for eachnote in notelist:
                    #out_put.write(eachnote.getstr()+'\n')
                    midstream.append(eachnote.getnote())
                
                s.append(midstream)
                notelist.clear()
            else:
                continue
    
    if len(notelist)!= 0:
        midstream = stream.Measure(number=measurenum)
        for eachnote in notelist:
            #out_put.write(eachnote.getstr()+'\n')
            midstream.append(eachnote.getnote())
        s.append(midstream)

        notelist.clear()
    if play_it == True:play(s)
    #out_put.close()
    return s


def str2tone(music_str):
    #out_put = open('out.txt','w')
    notelist = []
    s = stream.Score(id='mainScore')
    '''
    lowerflag  -  低音标志()
    higherflag -  高音标志[]
    shotterflag - 短时标志<>
    longgerflag - 长时标志--
    
    '''
    lowerflag = False
    higherflag = False
    sharpflag = False
    shotterflag = 0
    measurenum = 1
    left = ''
    right = ''
    for each in music_str:
        #print(each)
        if isnum(each):
            midnote = StrNote(int(each))
            if lowerflag == True:
                midnote.lower()
            if higherflag == True:
                midnote.higher()
            if shotterflag != 0:
                for i in range(shotterflag):
                    midnote.shotter()
            if sharpflag == True:
                midnote.setsharp()
                #print(midnote)
                sharpflag = False
            notelist.append(midnote)
        else:
            if each == '(':
                lowerflag = True
            elif each == ')':
                lowerflag = False
            elif each == '[':
                higherflag = True 
            elif each == ']':
                higherflag = False 
            elif each == '<':
                shotterflag += 1 
                left = '<'+left
            elif each == '>':
                shotterflag -= 1
                right = right+'>'
            elif each == '.':
                notelist[len(notelist)-1].adddot()
                right = right+'.'
            elif each == '-':
                notelist[len(notelist)-1].longer()
                right = right+'-'
            elif each == '#':
                #sharpflag = True
                notelist[len(notelist)-1].setsharp()
            elif each == 'b':
                notelist[len(notelist)-1].setflat()
    
    return left+notelist[0].getstr()+right


tone_list = ['1','2','3','4','5','6','7']

import random
'''
Summary:
    制作一个音调
'''
def MakeStrTone():
    # 随机选择音调
    tone = random.choice(tone_list) 
    
    # 是否高低
    # if random.randint(0, 1) == 0:
    
    if random.randint(0, 1) == 0:
        if random.randint(0, 1) == 0:
            tone = tone+'#'
        else:
            tone = tone+'b'
        
        
    # 没有被高音
    if has_high == False:
        # 添加高音
        if random.randint(0, 1) == 0:
            tone = '['+tone
            has_high = True
    # 没有被低音
    elif has_high == False and has_low == False:
    #if has_low == False:
        # 添加低音
        if random.randint(0, 1) == 0:
            tone = '('+tone
            has_low = True
    else:
        if has_high == True:
            # 去掉高音
            if random.randint(0, 1) == 0:
                tone = tone+']'
                has_high = False
        elif has_low == True:
            # 去掉低音
            if random.randint(0, 1) == 0:
                tone = tone+')'
                has_low = False       
    # 没有被加快
    if has_fast == False:
        # 
        if random.randint(0, 1) == 0:
            tone = '<'+tone
            has_fast = True
    # 被加快了
    else:
        # 降下来
        if random.randint(0, 1) == 0:
            tone = tone+'>'
            has_fast = False
    
    return tone, has_high, has_low, has_fast



'''
Summary:
    制作一个音调
'''
def MakeStrTone2():
    # 随机选择音调
    tone = random.choice(tone_list) 
    
    # 是否升降
    if random.randint(0, 1) == 0 and tone not in ['3','7']:
        pan = random.randint(0, 2)
        if pan == 0:
            tone = tone+'#'
        
    # 是否高低
    if random.randint(0, 1) == 0:
        pan = random.randint(0, 2)
        if pan == 0:
            tone = '['+tone+']'
        elif pan == 1:
            tone = '('+tone+')'
    
    # 是否延长/缩短
    if random.randint(0, 1) == 0:
        pan = random.randint(0, 1)
        # 延长
        if pan == 0:
            # 0是 1是- 2是-- 3是---
            pan = random.randint(0, 3)
            if pan == 1:
                tone =  tone + '-'
            elif pan == 2:
                tone =  tone + '--'
            elif pan == 3:
                tone =  tone + '---'
        else:
            tone = '<'+ tone + '>'
            pan = random.randint(0, 2)
            if pan == 0:
                tone = '<'+ tone + '>'
            elif pan == 1:
                tone = tone + '.'
                
        
    
    return tone
    
'''
Summary:
    制作一个谱子
'''
def MakeStrScore(tones_num=10):
    has_high,has_low, has_fast = False,False,False
    Score = ''
#     for i in range(tones_num):
#         tone, has_high, has_low, has_fast = MakeStrTone( has_high,has_low, has_fast)
#         Score += tone
#         if i%4 == 0:
#             Score += '|'
#     if has_high == True:
#         Score = Score+']'
#     if has_low == True:
#         Score = Score+')'
#     if has_fast == True:
#         Score = Score+'>'
#     print(Score)
    ScoreList = []
    for i in range(tones_num):
        mytone = MakeStrTone2() 
        Score += mytone
        #ScoreList.append(mytone)
        ScoreList.append(str2tone(mytone))
        if i%4 == 0:Score += '|'
    return Score,ScoreList


import datetime
import pickle
import time
import os
import shutil 


'''
Summary
    保存为音频数据
'''
def writemusic(s,path):
    #path = r'.\happy.mid' 
    s.write('midi', fp=path)
    

'''
Summary:
    制作数据集
Parameters:
    datas_num - 要制作的数据集的数量 int
'''
from shutil import copyfile,rmtree
def MakeDataSet(datas_num):
    nowtime = time.strftime("%Y-%m-%d_%H_%M_%S", time.localtime())
    dirname = nowtime[:-3]
    if os.path.exists('./wavDatasets/') == False:
        os.mkdir('./wavDatasets/')
    if os.path.exists('./midiDatasets/') == False:
        os.mkdir('./midiDatasets/')
        
    if os.path.exists('./wavDatasets/'+dirname):
        rmtree('./wavDatasets/'+dirname)
    if os.path.exists('./midiDatasets/'+dirname):
        rmtree('./midiDatasets/'+dirname)
    if os.path.exists('../wavDatasets/'):
        rmtree('../wavDatasets/')
    
    os.mkdir('./wavDatasets/'+dirname)
    os.mkdir('./midiDatasets/'+dirname)
    os.mkdir('../wavDatasets/')
    
    music_dic = {}
    for i in range(datas_num):
        string_s,string_list = MakeStrScore(20)
        file = nowtime+'_'+str(i)
        #print(file)
        s = str2stream(string_s,False)
        writemusic(s,'./midiDatasets/{0}/{1}.mid'.format(dirname,file))
        music_dic[file] = string_list
        #path = 'midiDatasets\'+ datetime.datetime.now().strftime('%Y-%m-%d')
    
   
    
    data_output = open('../wavDatasets/music_dic.pkl'.format(dirname),'wb')
    pickle.dump(music_dic,data_output)
    data_output.close()
    #print('java -jar midi2wav.jar ./midiDatasets/{0}'.format(dirname))
    os.system('java -jar midi2wav.jar ./midiDatasets/{0}'.format(dirname))
    for file in os.listdir('./midiDatasets/{0}'.format(dirname)):
        if file.find('.wav')>0: 
            copyfile('./midiDatasets/{0}/{1}'.format(dirname,file), './wavDatasets/{0}/{1}'.format(dirname,file))
            os.remove('./midiDatasets/{0}/{1}'.format(dirname,file))
    writesmall('./wavDatasets/{0}/'.format(dirname)) 
    return music_dic
#     import time

# # 格式化成2016-03-20 11:45:39形式
# print (time.strftime("%Y-%m-%d_%H:%M:%S", time.localtime()))
#     import pickle
# import numpy as np

# your_data = np.ones((4,5),np.uint8)
# # wb 以二进制写入
# data_output = open('data.pkl','wb')
# pickle.dump(your_data,data_output)
# data_output.close()

# # rb 以二进制读取
# data_input = opne('data.pkl','rb')
# read_data = pickle.load(data_input)
# data_input.close(）
def GetMusicDic():
    with open('../wavDatasets/music_dic.pkl', 'rb') as file:
        music_dic = pickle.load(file)
    return music_dic

each_feq = 44100/1024
# 最小的那个阈值
yu1 = 246 // each_feq
# 最大的那个阈值
yu2 = 2489 // each_feq
print(each_feq,yu1,yu2)


'''
Summary:
    间隔1000取样
'''
def writesmall(dirpath):
    print(dirpath)
    for file in os.listdir(dirpath):
        # 读取数据文件
        y, sr = librosa.load(dirpath+file,44100)
        # 得到短时傅里叶
        D = librosa.stft(y)
        # 进行处理
        for i in range(len(D)):
            if i < yu1 or i > yu2:
                D[i][D[i]< 1000] = 0  
        # 再转回来
        n = len(y)
        y_out = librosa.istft(D, length=n)
        # 这里进行采样
        index = np.arange(0, len(y_out), 1000)
        y_out2 = y_out[index]
        
        
        #librosa.display.waveplot(y_out, sr=sr)
        # 保存
        librosa.output.write_wav('../wavDatasets/'+file,y,44100)
        #librosa.output.write_wav('../wavDatasets/'+file,y_out2,44100)
        #pysndfile.sndio.write('../wavDatasets/'+file,y_out2, output_istft, rate=16000, format='wav', enc='pcm16')

43.06640625 5.0 57.0


In [43]:
test_note_list = []
str_note = ['1','2','3','4','5','6','7']
# 普通数据音调
for note in str_note:
    test_note_list.append(note)    
    # 升音
    if note not in ['3','7']:
        test_note_list.append(note+'#') 
    # 降音
    #test_note_list.append(note+'b')
    # 高音
    test_note_list.append('['+note+']')
    # 低音
    test_note_list.append('('+note+')')
    # 高音升音
    if note not in ['3','7']:
        test_note_list.append('['+note+'#]')
    # 高音降音
    #test_note_list.append('['+note+'b]')
    # 低音升音
    if note not in ['3','7']:
        test_note_list.append('('+note+'#)')
    # 低音降音
    #test_note_list.append('('+note+'b)')

length = len(test_note_list)
# 声音时值增加or减少
for i in range(length):
    note = test_note_list[i]
    # 增加-
    test_note_list.append(note+'-')
    # 增加--
    test_note_list.append(note+'--')
    # 增加---
    test_note_list.append(note+'---')
    # 增加.
    test_note_list.append(note+'.')
    # 增加<>
    test_note_list.append('<'+note+'>')
    # 增加<<>>
    test_note_list.append('<<'+note+'>>')
    
    

In [44]:
str(test_note_list)

"['1', '1#', '[1]', '(1)', '[1#]', '(1#)', '2', '2#', '[2]', '(2)', '[2#]', '(2#)', '3', '[3]', '(3)', '4', '4#', '[4]', '(4)', '[4#]', '(4#)', '5', '5#', '[5]', '(5)', '[5#]', '(5#)', '6', '6#', '[6]', '(6)', '[6#]', '(6#)', '7', '[7]', '(7)', '1-', '1--', '1---', '1.', '<1>', '<<1>>', '1#-', '1#--', '1#---', '1#.', '<1#>', '<<1#>>', '[1]-', '[1]--', '[1]---', '[1].', '<[1]>', '<<[1]>>', '(1)-', '(1)--', '(1)---', '(1).', '<(1)>', '<<(1)>>', '[1#]-', '[1#]--', '[1#]---', '[1#].', '<[1#]>', '<<[1#]>>', '(1#)-', '(1#)--', '(1#)---', '(1#).', '<(1#)>', '<<(1#)>>', '2-', '2--', '2---', '2.', '<2>', '<<2>>', '2#-', '2#--', '2#---', '2#.', '<2#>', '<<2#>>', '[2]-', '[2]--', '[2]---', '[2].', '<[2]>', '<<[2]>>', '(2)-', '(2)--', '(2)---', '(2).', '<(2)>', '<<(2)>>', '[2#]-', '[2#]--', '[2#]---', '[2#].', '<[2#]>', '<<[2#]>>', '(2#)-', '(2#)--', '(2#)---', '(2#).', '<(2#)>', '<<(2#)>>', '3-', '3--', '3---', '3.', '<3>', '<<3>>', '[3]-', '[3]--', '[3]---', '[3].', '<[3]>', '<<[3]>>', '(3)-', '

In [45]:
str2tone('1--')

'C5--'

In [46]:
test_ans_list = []
for note in test_note_list:
    test_ans_list.append(str2tone(note))

In [47]:
str(test_ans_list)

"['C5', 'C5#', 'C6', 'C4', 'C6#', 'C4#', 'D5', 'D5#', 'D6', 'D4', 'D6#', 'D4#', 'E5', 'E6', 'E4', 'F5', 'F5#', 'F6', 'F4', 'F6#', 'F4#', 'G5', 'G5#', 'G6', 'G4', 'G6#', 'G4#', 'A5', 'A5#', 'A6', 'A4', 'A6#', 'A4#', 'B5', 'B6', 'B4', 'C5-', 'C5--', 'C5---', 'C5.', '<C5>', '<<C5>>', 'C5#-', 'C5#--', 'C5#---', 'C5#.', '<C5#>', '<<C5#>>', 'C6-', 'C6--', 'C6---', 'C6.', '<C6>', '<<C6>>', 'C4-', 'C4--', 'C4---', 'C4.', '<C4>', '<<C4>>', 'C6#-', 'C6#--', 'C6#---', 'C6#.', '<C6#>', '<<C6#>>', 'C4#-', 'C4#--', 'C4#---', 'C4#.', '<C4#>', '<<C4#>>', 'D5-', 'D5--', 'D5---', 'D5.', '<D5>', '<<D5>>', 'D5#-', 'D5#--', 'D5#---', 'D5#.', '<D5#>', '<<D5#>>', 'D6-', 'D6--', 'D6---', 'D6.', '<D6>', '<<D6>>', 'D4-', 'D4--', 'D4---', 'D4.', '<D4>', '<<D4>>', 'D6#-', 'D6#--', 'D6#---', 'D6#.', '<D6#>', '<<D6#>>', 'D4#-', 'D4#--', 'D4#---', 'D4#.', '<D4#>', '<<D4#>>', 'E5-', 'E5--', 'E5---', 'E5.', '<E5>', '<<E5>>', 'E6-', 'E6--', 'E6---', 'E6.', '<E6>', '<<E6>>', 'E4-', 'E4--', 'E4---', 'E4.', '<E4>', '<<E4>

In [48]:
for i in range(len(test_note_list)):
    print(test_note_list[i],test_ans_list[i])

1 C5
1# C5#
[1] C6
(1) C4
[1#] C6#
(1#) C4#
2 D5
2# D5#
[2] D6
(2) D4
[2#] D6#
(2#) D4#
3 E5
[3] E6
(3) E4
4 F5
4# F5#
[4] F6
(4) F4
[4#] F6#
(4#) F4#
5 G5
5# G5#
[5] G6
(5) G4
[5#] G6#
(5#) G4#
6 A5
6# A5#
[6] A6
(6) A4
[6#] A6#
(6#) A4#
7 B5
[7] B6
(7) B4
1- C5-
1-- C5--
1--- C5---
1. C5.
<1> <C5>
<<1>> <<C5>>
1#- C5#-
1#-- C5#--
1#--- C5#---
1#. C5#.
<1#> <C5#>
<<1#>> <<C5#>>
[1]- C6-
[1]-- C6--
[1]--- C6---
[1]. C6.
<[1]> <C6>
<<[1]>> <<C6>>
(1)- C4-
(1)-- C4--
(1)--- C4---
(1). C4.
<(1)> <C4>
<<(1)>> <<C4>>
[1#]- C6#-
[1#]-- C6#--
[1#]--- C6#---
[1#]. C6#.
<[1#]> <C6#>
<<[1#]>> <<C6#>>
(1#)- C4#-
(1#)-- C4#--
(1#)--- C4#---
(1#). C4#.
<(1#)> <C4#>
<<(1#)>> <<C4#>>
2- D5-
2-- D5--
2--- D5---
2. D5.
<2> <D5>
<<2>> <<D5>>
2#- D5#-
2#-- D5#--
2#--- D5#---
2#. D5#.
<2#> <D5#>
<<2#>> <<D5#>>
[2]- D6-
[2]-- D6--
[2]--- D6---
[2]. D6.
<[2]> <D6>
<<[2]>> <<D6>>
(2)- D4-
(2)-- D4--
(2)--- D4---
(2). D4.
<(2)> <D4>
<<(2)>> <<D4>>
[2#]- D6#-
[2#]-- D6#--
[2#]--- D6#---
[2#]. D6#.
<[2#]> <D6#>

In [40]:
import json
test_dict  = {}
for i in range(len(test_note_list)):
    testname = 'test{0}'.format(i)
    print(test_note_list[i],test_ans_list[i])
    test_dict[testname] = {'note_str':test_note_list[i],'note_ans':test_ans_list[i]}
#dumps 将数据转换成字符串
json_str = json.dumps(test_dict,indent=4)# 使用indent格式化输出

1 C5
1# C5#
[1] C6
(1) C4
[1#] C6#
(1#) C4#
2 D5
2# D5#
[2] D6
(2) D4
[2#] D6#
(2#) D4#
3 E5
[3] E6
(3) E4
4 F5
4# F5#
[4] F6
(4) F4
[4#] F6#
(4#) F4#
5 G5
5# G5#
[5] G6
(5) G4
[5#] G6#
(5#) G4#
6 A5
6# A5#
[6] A6
(6) A4
[6#] A6#
(6#) A4#
7 B5
[7] B6
(7) B4
1- C5-
1-- C5--
1--- C5---
1. C5.
<1> <C5>
<<1>> <<C5>>
1#- C5#-
1#-- C5#--
1#--- C5#---
1#. C5#.
<1#> <C5#>
<<1#>> <<C5#>>
[1]- C6-
[1]-- C6--
[1]--- C6---
[1]. C6.
<[1]> <C6>
<<[1]>> <<C6>>
(1)- C4-
(1)-- C4--
(1)--- C4---
(1). C4.
<(1)> <C4>
<<(1)>> <<C4>>
[1#]- C6#-
[1#]-- C6#--
[1#]--- C6#---
[1#]. C6#.
<[1#]> <C6#>
<<[1#]>> <<C6#>>
(1#)- C4#-
(1#)-- C4#--
(1#)--- C4#---
(1#). C4#.
<(1#)> <C4#>
<<(1#)>> <<C4#>>
2- D5-
2-- D5--
2--- D5---
2. D5.
<2> <D5>
<<2>> <<D5>>
2#- D5#-
2#-- D5#--
2#--- D5#---
2#. D5#.
<2#> <D5#>
<<2#>> <<D5#>>
[2]- D6-
[2]-- D6--
[2]--- D6---
[2]. D6.
<[2]> <D6>
<<[2]>> <<D6>>
(2)- D4-
(2)-- D4--
(2)--- D4---
(2). D4.
<(2)> <D4>
<<(2)>> <<D4>>
[2#]- D6#-
[2#]-- D6#--
[2#]--- D6#---
[2#]. D6#.
<[2#]> <D6#>

In [41]:
with open(r"test_file2.json","w") as f:
     f.write(json_str)
     print("保存json完成")

保存json完成


In [49]:
str(test_ans_list)

"['C5', 'C5#', 'C6', 'C4', 'C6#', 'C4#', 'D5', 'D5#', 'D6', 'D4', 'D6#', 'D4#', 'E5', 'E6', 'E4', 'F5', 'F5#', 'F6', 'F4', 'F6#', 'F4#', 'G5', 'G5#', 'G6', 'G4', 'G6#', 'G4#', 'A5', 'A5#', 'A6', 'A4', 'A6#', 'A4#', 'B5', 'B6', 'B4', 'C5-', 'C5--', 'C5---', 'C5.', '<C5>', '<<C5>>', 'C5#-', 'C5#--', 'C5#---', 'C5#.', '<C5#>', '<<C5#>>', 'C6-', 'C6--', 'C6---', 'C6.', '<C6>', '<<C6>>', 'C4-', 'C4--', 'C4---', 'C4.', '<C4>', '<<C4>>', 'C6#-', 'C6#--', 'C6#---', 'C6#.', '<C6#>', '<<C6#>>', 'C4#-', 'C4#--', 'C4#---', 'C4#.', '<C4#>', '<<C4#>>', 'D5-', 'D5--', 'D5---', 'D5.', '<D5>', '<<D5>>', 'D5#-', 'D5#--', 'D5#---', 'D5#.', '<D5#>', '<<D5#>>', 'D6-', 'D6--', 'D6---', 'D6.', '<D6>', '<<D6>>', 'D4-', 'D4--', 'D4---', 'D4.', '<D4>', '<<D4>>', 'D6#-', 'D6#--', 'D6#---', 'D6#.', '<D6#>', '<<D6#>>', 'D4#-', 'D4#--', 'D4#---', 'D4#.', '<D4#>', '<<D4#>>', 'E5-', 'E5--', 'E5---', 'E5.', '<E5>', '<<E5>>', 'E6-', 'E6--', 'E6---', 'E6.', '<E6>', '<<E6>>', 'E4-', 'E4--', 'E4---', 'E4.', '<E4>', '<<E4>

In [27]:
len(a)

63

In [26]:
import pytest
import json

# 读取数据
testname_list = []
for i in range(len(test_ans_list)):
    testname_list.append('test{0}'.format(i))
json_file = open("test_file.json",'r',encoding='gbk')  
a = json.load(json_file)

# 进行测试
@pytest.mark.parametrize('test_name', testname_list)
def test_str2note(test_name):
    print(a[test_name])

In [6]:
import librosa
import numpy as np
import os

In [16]:
'''
Summary:
    调用该函数返回最终的note列表

Return:
    data - note列表
'''
def ChangeMuisc2Note(filpath):
    y,rate=librosa.load(filpath,44100)
    #短时傅里叶
    fft=librosa.stft(y,n_fft=1024*2)
    #转换为分贝值
    D=librosa.amplitude_to_db(abs(fft),ref=np.max)
    #数据>0
    D=D+80
    
    data=music2note(D,rate/2)
    return data

music_rate = {}
music_data = []

path='wavDatasets//'
listdir=os.listdir(path)
data={}
for x in listdir:
    if 'wav' in x:
        y,rate=librosa.load('wavDatasets//'+x,44100)
        
        #短时傅里叶
        fft=librosa.stft(y,n_fft=1024*2)
        #转换为分贝值
        data=librosa.amplitude_to_db(abs(fft),ref=np.max)
        #数据>0
        data=data+80

        #转置 降噪
        data=np.transpose(data)
        data=data/data.max()
        data[data<0.75]=0
        #单位频率
        unit_rate=rate/(data.shape[1]-1)
        music_rate[x] = unit_rate

        for each in data:
            music_data.append(np.array(each))
    
    

In [18]:
unit_rate

43.06640625

In [12]:
len(music_data['2020-05-29_23_00_43_0.wav'])

1292

In [27]:

unit_rate = 43.06640625
def music2note1(data,unit_rate):
    
    index = (data!=0).argmax(axis=0)
    return librosa.hz_to_note((index+1)*unit_rate)


def music2note2(i,rate):
    note = []
    for j in range(len(i)):
        #第一个非0值
        if i[j]>0 and len(note)==0:
            #频率转换音符
            note_str = librosa.hz_to_note((j+1)*unit_rate)
            note.append(note_str)
    if len(note)==1:
        return note[0]
    return  None

In [28]:
for each in music_data:
    print(music2note1(each,unit_rate) == music2note2(each,unit_rate))

False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
T

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
Tr

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
Tru

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
Tru

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
T

KeyboardInterrupt: 

In [32]:
max(music_data[0])
(music_data[0]!=0).argmax(axis=0)

0

In [37]:
import json
import random
test_dict  = {}
music_data2 = random.sample(music_data,100)
for i in range(len(music_data2)):
    testname = 'test{0}'.format(i)
    test_dict[testname] = {'music_data':music_data2[i].tolist(),'unit_rate':43.06640625}
#dumps 将数据转换成字符串
json_str = json.dumps(test_dict,indent=4)# 使用indent格式化输出
with open(r"C:\Users\UncleDong\Desktop\TAPD 所有工程\测试用例\compare_file1.json","w") as f:
     f.write(json_str)
     print("保存json完成")

保存json完成


In [42]:
folat

NameError: name 'folat' is not defined

In [ ]:
import pytest
import json

# 读取数据
testname_list = []
for i in range(len(test_ans_list)):
    testname_list.append('test{0}'.format(i))
    
    
    
json_file = open("test_file.json",'r',encoding='gbk')  
a = json.load(json_file)

# 进行测试
@pytest.mark.parametrize('test_name', testname_list)
def test_str2note(test_name):
    print(a[test_name])